In [ ]:
from skimage.io import imread
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import time

In [ ]:
rootfolder = ".."

Useful function for plot a 2D dictionary


In [ ]:
def get_dictionary_img(D):
    M, N = D.shape
    p = int(round(np.sqrt(M)))
    nnn = int(np.ceil(np.sqrt(N)))
    bound = 2
    img = np.ones((nnn * p + bound * (nnn - 1), nnn * p + bound * (nnn - 1)))
    for i in range(N):
        m = np.mod(i, nnn)
        n = int((i - m) / nnn)
        m = m * p + bound * m
        n = n * p + bound * n
        atom = D[:, i].reshape((p, p))
        if atom.min() < atom.max():
            atom = (atom - atom.min()) / (atom.max() - atom.min())
        img[m : m + p, n : n + p] = atom

    return img

Define a function that implements the OMP


In [ ]:
def OMP(s, D, L, tau):
    _, N = D.shape
    r = s.copy()  # initial residual
    omega = []  # support set
    x_OMP = np.zeros(N)  # final sparse code

    while len(omega) < L and np.linalg.norm(r) > tau:
        # SWEEP STEP: compute correlations between residual and dictionary atoms
        e = np.zeros(N)
        for j in range(N):
            e[j] = D[:, j].T @ r

        # find the column index with maximum correlation
        jStar = np.argmax(np.abs(e))

        # UPDATE support set
        if jStar not in omega:
            omega.append(jStar)

        # update coefficients using least squares
        D_omega = D[:, omega]
        x_omega, _, _, _ = np.linalg.lstsq(D_omega, s, rcond=None)

        # update residual
        r = s - D_omega @ x_omega

    # construct full sparse vector
    for i, idx in enumerate(omega):
        x_OMP[idx] = x_omega[i]

    return x_OMP

## Dictionary Learning


Load the image and rescale it in $[0,1]$


In [ ]:
path_image = (
    f"{rootfolder}/data/barbara.png"  #  barbara.png, cameraman.png, Lena512.png
)

img = imread(path_image) / 255

imsz = img.shape

# patch size
p = 8

# number of elements in the patch
M = p**2

Extract a bunch of random patches from the image and build the training set $S$


In [ ]:
npatch = 10000

S = np.zeros((M, npatch))
# S =

Remove the mean from the patches (each column of $S$ must have zero-mean)


In [ ]:
# S =


Define a function that implements the KSVD


In [ ]:
def ksvd(S, M, N, max_iter, npatch, L, print_time=False):

    # intialize the dictionary
    # D =

    # normalize each column of D (zero mean and unit norm)
    # UPDATE D

    # initialize the coefficient matrix
    X = np.zeros((N, npatch))

    # Main KSVD loop
    for iter in range(max_iter):
        time_start = time.time()

        # Sparse coding step
        # perform the sparse coding via OMP of all the columns of S
        for n in range(npatch):
            # X[:, n] =

        # Dictionary update step
        # iterate over the columns of D
        for j in range(N):
            # find which signals uses the j-th atom in the sparse coding
            # omega =

            if len(omega) == 0:
                # if the atom is never used then ignore or substitute it with a random vector
            else:
                # compute the residual matrix E, ignoring the j-th atom
                # E =

                # restrict E to the columns indicated by omega
                # Eomega =

                # Compute the best rank-1 approximation
                # U, Sigma, V =

                # update the dictionary
                # D

                # update the coefficient matrix
                # X

        time_end = time.time()
        if print_time:
            print(f'Iteration {iter} runtime: {time_end-time_start}')

    return D


In [ ]:
# number of columns of the dictionary
N = 256

# number of iteration of the KSVD
max_iter = 10

# maximum number of nonzero coefficients for the sparse coding
L = 4


# Call the KSVD implementation
# D =

Show the learned dictionary


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].imshow(img, cmap="gray")
ax[0].set_title(f"Image {path_image.split('/')[-1]}")
ax[0].axis("off")

img_dict = get_dictionary_img(D)
ax[1].imshow(img_dict, cmap="gray")
ax[1].set_title(f"Dictionary learned from {path_image.split('/')[-1]}")
ax[1].axis("off")

plt.tight_layout()

## OMP denoising with learned dictionaries


In [ ]:
img_clean = imread(f"{rootfolder}/data/barbara.png") / 255

# Corrupte the image

sigma_noise = 20 / 255
# noisy_img =

In [ ]:
# psnr_noisy =

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img_clean, cmap="gray")
ax[0].set_title(f"Original image (barbara.png)")
ax[0].axis("off")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")
ax[1].axis("off")

plt.tight_layout()


In [ ]:
# patch size
p = 8

# number of elements in the patch
M = p**2

# number of columns of the dictionary
N = 256

# number of iteration of the KSVD
max_iter = 10

# maximum number of nonzero coefficients for the sparse coding
L = 4

Generic dictionary


In [ ]:
D_generic = loadmat(f"{rootfolder}/data/dict_nat_img.mat")["D"]

Dictionary learned from a different image


In [ ]:
img = imread(f"{rootfolder}/data/cameraman.png") / 255

# Extract random patches
npatch = 10000

S = S - np.mean(S, axis=0, keepdims=True)

# S =

# Learn the dictionary
# D_diff =

Dictionary learned from the noisy image


In [ ]:
# Extract random patches
npatch = 10000

# S =


# Learn the dictionary
# D_noisy =

Dictionary learned from the clean image


In [ ]:
# Extract random patches
npatch = 10000

# S = S - np.mean(S, axis=0, keepdims=True)


# Learn the dictionary
# D_clean =

OMP denoising


In [ ]:
def omp_denoising(noisy_img, D, step, tau):
    pass

Denoising using the learned dictionaries


In [ ]:
# set the threshold
tau = 1.15 * p * sigma_noise

# define the step (=p for non overlapping paches)
STEP = 4  # STEP = 1 might be very time consuming, start with larger STEP

Solve the four denoising problems


In [ ]:
# Denoising with dictionary D_generic
# img_hat_generic =

# Denoising with dictionary D_diff
# img_hat_diff =

# Denoising with dictionary D_noisy
# img_hat_noisy =

# Denoising with dictionary D_clean
# img_hat_clean =

Visualize the results


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(get_dictionary_img(D_generic), cmap="gray")
ax[0].set_title("Dictionary (Generic)")
ax[0].axis("off")

# psnr_hat =

ax[1].imshow(img_hat_general, cmap="gray")
ax[1].set_title(f"Denoised image, PSNR = {psnr_hat:.2f}")
ax[1].axis("off")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(get_dictionary_img(D_diff), cmap="gray")
ax[0].set_title("Dictionary (From a different image)")
ax[0].axis("off")

# psnr_hat =

ax[1].imshow(img_hat_diff, cmap="gray")
ax[1].set_title(f"Denoised image, PSNR = {psnr_hat:.2f}")
ax[1].axis("off")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(get_dictionary_img(D_noisy), cmap="gray")
ax[0].set_title("Dictionary (From the noisy image)")
ax[0].axis("off")

# psnr_hat =

ax[1].imshow(img_hat_noisy, cmap="gray")
ax[1].set_title(f"Denoised image, PSNR = {psnr_hat:.2f}")
ax[1].axis("off")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(get_dictionary_img(D_clean), cmap="gray")
ax[0].set_title("Dictionary (from the clean image)")
ax[0].axis("off")

# psnr_hat =

ax[1].imshow(img_hat_clean, cmap="gray")
ax[1].set_title(f"Denoised image, PSNR = {psnr_hat:.2f}")
ax[1].axis("off")

plt.tight_layout()